<a href="https://colab.research.google.com/github/zuhaib786/TF_Learning_Notebooks/blob/main/First_Kaggle_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cat vs Dog Classifier
My first implementation of a cat vs dog classifier

In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import torch
from matplotlib import pyplot as plt
import os


In [2]:

os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!kaggle competitions download -c dogs-vs-cats

100% 812M/812M [00:20<00:00, 43.9MB/s]
100% 812M/812M [00:20<00:00, 41.3MB/s]


In [3]:
!unzip dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [4]:
!unzip -qq train.zip -d data

In [5]:
!mkdir /content/data/cats
!mkdir /content/data/dogs

In [6]:
source = '/content/data/train'
cat_dest = '/content/data/cats'
dog_dest = '/content/data/dogs'

In [7]:

files = os.listdir('/content/data/train')
files[0]


'dog.9876.jpg'

In [8]:
def getClass(filename):
    filename = filename[0:3]
    if filename == 'cat':
        return 0
    return 1


In [9]:
for f in files:
  src_path = os.path.join(source, f)
  if getClass(f) == 0:
    dest_path = os.path.join(cat_dest, f)
  else:
    dest_path = os.path.join(dog_dest, f)
  os.rename(src_path, dest_path)

In [10]:
!mkdir data/valid
!mkdir data/train/cats
!mkdir data/train/dogs
!mkdir data/valid/dogs
!mkdir data/valid/cats

In [11]:
cat_dest_train = '/content/data/train/cats'
cat_dest_valid = '/content/data/valid/cats'
dog_dest_train = '/content/data/train/dogs'
dog_dest_valid = '/content/data/valid/dogs'


In [12]:
cat_files = os.listdir(cat_dest)
dog_files = os.listdir(dog_dest)
for i, id in enumerate(cat_files):
  if i >=12000:
    dest = os.path.join(cat_dest_valid, id)
    source = os.path.join(cat_dest, id)
  else:
    dest = os.path.join(cat_dest_train, id)
    source =os.path.join(cat_dest, id)
  os.rename(source, dest)
for i, id in enumerate(dog_files):
  if i >=12000:
    dest = os.path.join(dog_dest_valid, id)
    source = os.path.join(dog_dest, id)
  else:
    dest = os.path.join(dog_dest_train, id)
    source =os.path.join(dog_dest, id)
  os.rename(source, dest)

In [13]:
import keras
from keras.models import Sequential
from keras.layers import   Conv2D, Activation, Dropout, MaxPooling2D, Flatten, Dense
from keras import regularizers, optimizers
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      896       
                                                                 
 activation (Activation)     (None, 100, 100, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 98, 98, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 98, 98, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 49, 49, 32)        0         
                                                        

In [15]:
from keras_preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_directory(
    directory = 'data/train',
    class_mode= 'binary', 
    batch_size=8,
    target_size = (100, 100), 
    shuffle=True
)
valid_generator = test_datagen.flow_from_directory(
    directory = 'data/valid',
    class_mode = 'binary',
    target_size = (100, 100),
    batch_size = 8,
    shuffle = False
)

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [16]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [17]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10


<ipython-input-17-7e17ccb8d9c1>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


3000/3000 [==============================] - 97s 29ms/step - loss: 0.5864 - accuracy: 0.6777 - val_loss: 0.5208 - val_accuracy: 0.7290
Epoch 2/10
3000/3000 [==============================] - 83s 28ms/step - loss: 0.5026 - accuracy: 0.7545 - val_loss: 0.4788 - val_accuracy: 0.7740
Epoch 3/10
3000/3000 [==============================] - 83s 28ms/step - loss: 0.4674 - accuracy: 0.7809 - val_loss: 0.4445 - val_accuracy: 0.8000
Epoch 4/10
3000/3000 [==============================] - 85s 28ms/step - loss: 0.4322 - accuracy: 0.8035 - val_loss: 0.4431 - val_accuracy: 0.7980
Epoch 5/10
3000/3000 [==============================] - 83s 28ms/step - loss: 0.4070 - accuracy: 0.8181 - val_loss: 0.4179 - val_accuracy: 0.8140
Epoch 6/10
3000/3000 [==============================] - 85s 28ms/step - loss: 0.3934 - accuracy: 0.8294 - val_loss: 0.4158 - val_accuracy: 0.8190
Epoch 7/10
3000/3000 [==============================] - 82s 27ms/step - loss: 0.3816 - accuracy: 0.8346 - val_loss: 0.4255 - val_accura

In [ ]:
!unzip -qq test1.zip

In [19]:
len(os.listdir('test1'))

12500

Found 0 images belonging to 0 classes.


In [22]:
files = os.listdir('test1')
files[0]


'5219.jpg'

In [23]:
!mkdir test1/all

In [24]:
source = '/content/test1'
dest = '/content/test1/all'
for file in files:
  source_name = os.path.join(source, file)
  dest_name = os.path.join(dest, file)
  os.rename(source_name, dest_name)

In [26]:
os.listdir('test1')

['all']

In [29]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_directory(
directory = '/content/test1',
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(100,100))

Found 12500 images belonging to 1 classes.


In [30]:
pred=model.predict_generator(test_generator,
verbose=1)

<ipython-input-30-2fbec8efda99>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_generator,


12500/12500 [==============================] - 54s 4ms/step


In [32]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [33]:
predictions = pred > 0.5

In [36]:
predictions = predictions.astype(int)
predictions[0]

array([1])

In [ ]:
preds = predictions.tolist()
preds

In [41]:
data = []
for pred in preds:
  data.append(pred[0])
Di = {'id' : list( range(1,  len(preds) + 1)), 'label' : data}
df = pd.DataFrame.from_dict(Di)
df.head()

,id,label
0,1,1
1,2,0
2,3,0
3,4,1
4,5,1


In [42]:
df.to_csv('sub.csv', index = False)